In [4]:
def classify_category(category: str) -> str:
    """
    Classify the given category as either "E-commerce" or "Classified"
    based on predefined criteria.

    Parameters:
    category (str): The product or service category to classify.

    Returns:
    str: The classification result ("E-commerce" or "Classified").
    """

    PROMPT_TEMPLATE = f"""
    Role: Act as an expert marketplace analyst with specialized knowledge in category management for platforms like BDstall.com.

    Objective: Analyze the provided list of product and service categories. For each item, classify it as either "E-commerce" or "Classified" based on the detailed criteria below.

    Primary Classification Mandate:
    The classification must be based on the item's physical size, typical purchase frequency (Lifetime Value - LTV), and the nature of the transaction (direct purchase vs. service/negotiation-based).

    Classification Criteria:

    1. Classify as "Classified" if the item meets ANY of the following conditions:
    * Nature: It is a service, rental, real estate, or a job posting.
    * Size & Value: It is a large, high-value item that typically requires physical inspection, delivery logistics, or installation (e.g., heavy machinery, vehicles, large appliances).
    * LTV (Lifetime Value): It is a one-time or very infrequent purchase for the average customer (LTV is approximately 1).
    * Keywords: Rent, Service, Repair, Land, Flat, Apartment, Vehicle, Car, Generator, Lift, Elevator, Installation, Used (for large items), Job, Tour.
    * Examples: Refrigerator, Air Conditioner, Generator, Elevator, Car, Land, Apartment for Rent, AC Repair Service, Web Design Service, Used Furniture.

    2. Classify as "E-commerce" if the item meets ALL of the following conditions:
    * Nature: It is a tangible product that can be easily packaged and shipped.
    * Size & Value: It is typically small to medium in size and suitable for a standard "add-to-cart" online shopping experience.
    * LTV (Lifetime Value): It has the potential for repeat purchases or a higher customer lifetime value (e.g., fashion, accessories, consumables, gadgets).
    * Keywords: Mobile, Laptop, Watch, Trimmer, Cosmetics, Shirt, Belt, Accessories, RAM, Keyboard, Mouse, Headphone, Book.
    * Examples: Smartwatch, Trimmer, Cosmetics, T-Shirt, Belt, Laptop Accessories, RAM, Mobile Phone, Office Chair.

    Input Format:
    You will receive a single string as category input.
    Now evaluate the whether it is "Classified" or "E-commerce" based on the above criteria.

    Output Format (Strict JSON):
    {
        "Classified": "YES / NO", 
        "E-commerce": "YES / NO"
    }

    Task:
    Now, process the category: "{category}"
    """

    return PROMPT_TEMPLATE

In [ ]:
import pandas as pd

input_file_location = "D:\\My Learning\\Machine-Learning-AI\\Full Stack AI Engineering\\Day 03\\bdstall_item_categories.csv"

df = pd.read_csv(input_file_location) 

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575 entries, 0 to 574
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Serial No.     575 non-null    int64  
 1   Category Name  70 non-null     object 
 2   Item List      565 non-null    object 
 3   Classified     0 non-null      float64
 4   E-commerce     0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 22.6+ KB


In [ ]:
df.columns = df.columns.str.strip()
all_item_list = df['Item List']

In [ ]:
# iterates over all non-empty items in the 'Item List' column
def iterate_all_items():
    non_empty_items = df['Item List'].dropna().apply(lambda x: x.strip() if isinstance(x, str) else x)
    non_empty_items = non_empty_items[non_empty_items != '']

    # Iterate over the non-empty values
    cnt = 0
    for item in non_empty_items:
        cnt += 1
        print(item)

    print(f"Total non-empty items: {cnt}")

## Configure the LLM (Groq)

In [ ]:
def get_ai_classification(prompt) -> str:

    from groq import Groq

    client = Groq()
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[
        {
            "role": "user",
            "content": classify_category
        }
        ],
        temperature=1,
        max_completion_tokens=1024,
        top_p=1,
        stream=True,
        stop=None
    )

    for chunk in completion:
        print(chunk.choices[0].delta.content or "", end="")
